Okay, here we are introducing the new custom loss function, which is the MLE. We need this to work before we proceed to the work with DeepCDR. The current custom loss we have over here are not working, so we may have to work on getting the loss function to accurately work. We may also need to adjust things like the computation of the r_square_values, as the previous code was not correctly computing these quantities.

In [1]:
import numpy as np
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split

2025-03-14 10:22:53.866636: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-14 10:22:54.220161: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-14 10:22:54.220219: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-14 10:22:54.307691: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-14 10:22:54.429190: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
# Step 1: Generate random data (10 features, 1 output)
n_samples = 1000
n_features = 10

In [4]:
# Generate random input features (X) and output (y)
X = np.random.randn(n_samples, n_features)
y = 2 * np.sum(X, axis=1) + np.random.randn(n_samples)  # Simple linear relation + noise

# Split the data into training, validation, and test sets (80% train, 10% validation, 10% test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [5]:
# Step 2: Define a simple fully connected neural network for Phase I (bootstrap models)
def create_bootstrap_nn(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

In [6]:
# Step 3: Define the noise variance estimation network (Phase II)
def create_noise_variance_nn(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)  # Output layer for variance prediction
    ])
    return model

In [7]:
# Step 4: Train bootstrap neural networks (Phase I)
def train_bootstrap_nns(X_train, y_train, B=10):
    bootstrap_models = []
    bootstrap_predictions = []

    # Generate B bootstrap samples and train models
    for _ in range(B):
        # Create a bootstrap sample by sampling with replacement
        indices = np.random.choice(len(X_train), size=len(X_train), replace=True)
        X_bootstrap = X_train[indices]
        y_bootstrap = y_train[indices]

        # Create and train a new model
        model = create_bootstrap_nn(X_train.shape[1:])
        model.fit(X_bootstrap, y_bootstrap, epochs=50, batch_size=32, verbose=0)
        bootstrap_models.append(model)

        # Store predictions on the original training data
        predictions = model.predict(X_train)
        bootstrap_predictions.append(predictions)

    return bootstrap_models, np.array(bootstrap_predictions)

In [8]:
# Step 7: Train the models
# Train the bootstrap models (Phase I)
bootstrap_models, bootstrap_predictions = train_bootstrap_nns(X_train, y_train, B=10)

2025-03-14 10:23:08.838496: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0
2025-03-14 10:23:10.820859: I external/local_xla/xla/service/service.cc:168] XLA service 0x151e65b39020 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-03-14 10:23:10.820911: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100S-PCIE-32GB, Compute Capability 7.0
2025-03-14 10:23:10.874510: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-03-14 10:23:10.992059: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1741965791.351297  286532 device_compiler.h:186] Compiled cluster using XLA!  Thi

25/25 [==============================] - 0s 925us/step


In [9]:
bootstrap_predictions.shape

(10, 800, 1)

In [10]:
np.squeeze(np.mean(bootstrap_predictions, axis = 0)).shape

(800,)

In [11]:
# The shape matches what we had earlier - no issue there

In [12]:
# function to get the variances according to the equation 6 in the paper
def equation_6_model_variance(all_preds):
    all_vars = []
    for i in range(all_preds.shape[1]):
        var = (1/(all_preds.shape[0]  - 1))*np.sum(np.square(all_preds[:,i,:] - np.mean(all_preds[:,i,:])))
        all_vars.append(var)

    return np.array(all_vars)

In [13]:
# Let's assume that we have already computed the r_sqaure values for the use in the second network with the function below

In [14]:
# Compute r^2(x_i) for each bootstrap model
def compute_r_squared(y_true, y_pred, model_variance):
    # reshape the predictions to (800,) as they are of shape (800,1)
    y_pred = np.squeeze(y_pred)
    residuals = (y_true - y_pred) ** 2 - model_variance
    return np.maximum(residuals, 0)

In [15]:
# Define the custom loss function as described in equation 12
def correct_custom_loss(r_true, r_pred):
    # first term in equation 12
    term_1 = tf.math.log(r_pred + 1)
    # define the second term
    term_2 = r_true/r_pred
    # cost function
    cost = 0.5 * tf.reduce_mean(term_1 + term_2)

    return cost

In [16]:
# Train function
def train_model_nne(X_train, y_train, bootstrap_predictions, model_variance, model):
    # Define custom loss inside model.compile using lambda
    bootstrap_mean_predictions = np.squeeze(np.mean(bootstrap_predictions, axis = 0))
    model.compile(optimizer='adam', 
                  loss=lambda y_true, y_pred: correct_custom_loss(
                      y_true, y_pred))  # y_true corresponds to r_true, and y_pred corresponds to r_pred
    
    # The true residuals (r_true) can be computed outside and passed during training
    r_true = compute_r_squared(y_train, bootstrap_mean_predictions, model_variance)
    
    # Fit the model with the true residuals
    model.fit(X_train, r_true, epochs=50, batch_size=32)

    return model

In [17]:
nn_e_model = create_noise_variance_nn(X_train.shape[1:])

In [18]:
model_variance_computed = equation_6_model_variance(bootstrap_predictions)

In [19]:
model_variance_computed.shape, y_train.shape, np.squeeze(np.mean(bootstrap_predictions, axis = 0)).shape

((800,), (800,), (800,))

In [20]:
# Train the noise variance estimation model (Phase II) using the custom likelihood-based loss function
nn_e_custom = train_model_nne(X_train, y_train, bootstrap_predictions, model_variance_computed, nn_e_model)

Epoch 1/50
25/25 [==============================] - 1s 3ms/step - loss: nan
Epoch 2/50
25/25 [==============================] - 0s 3ms/step - loss: 0.5259
Epoch 3/50
25/25 [==============================] - 0s 3ms/step - loss: 0.6426
Epoch 4/50
25/25 [==============================] - 0s 3ms/step - loss: 1.3783
Epoch 5/50
25/25 [==============================] - 0s 3ms/step - loss: 0.6671
Epoch 6/50
25/25 [==============================] - 0s 3ms/step - loss: 0.6627
Epoch 7/50
25/25 [==============================] - 0s 3ms/step - loss: 0.6620
Epoch 8/50
25/25 [==============================] - 0s 3ms/step - loss: 0.6614
Epoch 9/50
25/25 [==============================] - 0s 3ms/step - loss: 0.6609
Epoch 10/50
25/25 [==============================] - 0s 3ms/step - loss: 0.6603
Epoch 11/50
25/25 [==============================] - 0s 3ms/step - loss: 0.6597
Epoch 12/50
25/25 [==============================] - 0s 3ms/step - loss: 0.6589
Epoch 13/50
25/25 [==============================] -

In [21]:
# Step 8: Evaluate the models on the test set
# Make predictions with the bootstrap models
bootstrap_preds_test = np.array([model.predict(X_test) for model in bootstrap_models])

4/4 [==============================] - 0s 1ms/step


In [22]:
# Calculate the mean prediction across all bootstrap models
bootstrap_mean_preds_test = np.mean(bootstrap_preds_test, axis=0)

In [23]:
bootstrap_mean_preds_test.shape

(100, 1)

In [24]:
# Estimate the noise variance using NNₑ
predicted_variance_test = nn_e_custom.predict(X_test)

4/4 [==============================] - 0s 1ms/step


In [25]:
predicted_variance_test.shape

(100, 1)

In [26]:
# Print results
print(f"Mean prediction for test set: {np.mean(bootstrap_mean_preds_test)}")
print(f"Estimated noise variance (NNₑ) for test set: {np.mean(predicted_variance_test)}")

Mean prediction for test set: -0.620664119720459
Estimated noise variance (NNₑ) for test set: 1.0544947385787964


In [27]:
# Okay, now this is a wokring code, we jus need to ensure that whatever this does is indeed correct. Let's see if we can verify this

In [28]:
# Okay, now moving to computation of confidence intervals

In [29]:
# now the predictions are given by the first network, however, we need to compute the total variance for the test datapoints inorder to compute the prediction intervals.

In [30]:
# We need the test set, and all computations we will have from here will be for the test set. We also need tot accomodate the validation set in the code before we can say we have a working code for the toy dataset

In [31]:
# Predicted values from bootstraps for the test set - we computed this earlier

In [32]:
bootstrap_mean_preds_test.shape

(100, 1)

In [33]:
# we also have the error variance from the NNe network computed for the test set, gievn by 

predicted_variance_test.shape

(100, 1)

In [34]:
np.min(predicted_variance_test), np.max(predicted_variance_test)

(0.33393255, 2.1977303)

In [35]:
# Okay, so we do not have any variances that are zeros

In [36]:
# now what about the model misspecification variance?

# we can compute this based on the the function we defined earlier

In [37]:
model_variance_test = equation_6_model_variance(bootstrap_preds_test)

In [38]:
model_variance_test.shape

(100,)

In [39]:
# what are the minimum and the maximum values here?
np.min(model_variance_test), np.max(model_variance_test)

(0.025676732261975605, 0.7574890984429253)

In [40]:
# Okay, seems like this error variance is much more smaller compared to the error variance

In [41]:
# Let's commit this for now, and continue with this tomorrow.